In [128]:
import torch
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder

from PIL import Image

#train directory
data_dir = "../input/cat-dog-breedz/train_1"


#load the train data
dataset = ImageFolder(data_dir,transform = transforms.Compose([
    transforms.Resize((150,150)),transforms.ToTensor()
]))

In [130]:
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split

batch_size = 10
val_size = 5273 # 80:20 split 
train_size = len(dataset) - val_size 

train_data,val_data = random_split(dataset,[train_size,val_size])
print(f"Length of Train Data : {len(train_data)}")
print(f"Length of Validation Data : {len(val_data)}")

#load the train and validation into batches.
train_dl = DataLoader(train_data, batch_size, shuffle = True, num_workers = 4, pin_memory = True)
val_dl = DataLoader(val_data, batch_size*2, num_workers = 4, pin_memory = True)

Length of Train Data : 21093
Length of Validation Data : 5273


/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [131]:
import torch.nn as nn
import torch.nn.functional as F

# Preparing base for our model

class ImageClassificationBase(nn.Module):
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

In [132]:
class BreedClassification(ImageClassificationBase):
    # defining the main model - 4 layers xd
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            
            nn.Conv2d(3, 32, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2,2),
        
            nn.Conv2d(32, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.MaxPool2d(2,2),
            
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(0.6),
            nn.MaxPool2d(2,2),
            
            nn.Flatten(),
            nn.Linear(41472, 512),
            nn.ReLU(),
            nn.Linear(512,12)
        )
    
    def forward(self, xb):
        return self.network(xb)

In [133]:
model = BreedClassification()

In [134]:
# GPU stuff required for training on GPU
def get_default_device():
    """ Set Device to GPU or CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    

def to_device(data, device):
    "Move data to the device"
    if isinstance(data,(list,tuple)):
        return [to_device(x,device) for x in data]
    return data.to(device,non_blocking = True)

class DeviceDataLoader():
    """ Wrap a dataloader to move data to a device """
    
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
    
    def __iter__(self):
        """ Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b,self.device)
            
    def __len__(self):
        """ Number of batches """
        return len(self.dl)



In [135]:
device = get_default_device()
device

device(type='cuda')

In [136]:
# load the into GPU
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)
to_device(model, device)

BreedClassification(
  (network): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(32, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Dropout(p=0.5, inplace=False)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.6, inplace=False)
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (13): Flatten(start_dim=1, end_dim=-1)
    (14): Linear(in_features=41472, out_features=512, bias=True)
    (15): ReLU()
    (16): Linear(in_features=512, out_features=12, bias=True)
  )
)

In [137]:
#defining accuracy function

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

#eval function
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

#main fit function
def fit(epochs, lr, model, train_loader, val_loader, opt_func = torch.optim.SGD):
    
    history = []
    optimizer = opt_func(model.parameters(),lr)
    for epoch in range(epochs):
        # forward
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            #backward
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    
    return history



In [138]:
model = to_device(BreedClassification(),device)

In [140]:
#set the no. of epochs, optimizer funtion and learning rate
num_epochs = 5
opt_func = torch.optim.Adam
lr = 1e-4

In [141]:
#fitting the model on training data and record the result after each epoch
history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func)

Epoch [0], train_loss: 1.8084, val_loss: 2.0920, val_acc: 0.4114
Epoch [1], train_loss: 1.4188, val_loss: 1.9161, val_acc: 0.3949
Epoch [2], train_loss: 1.1774, val_loss: 1.7862, val_acc: 0.3966
Epoch [3], train_loss: 0.9775, val_loss: 1.6029, val_acc: 0.4527
Epoch [4], train_loss: 0.7977, val_loss: 1.6842, val_acc: 0.3998


In [142]:
def predict(image_path):
    img = Image.open(image_path)
    img = transforms.functional.resize(img, (150,150))
    img = transforms.ToTensor()(img)
    
    dogs = ["pug",
    "pitbull",
    "chihuahua",
    "golden_retriever",
    "german_shepherd",
    "komondor"] 
    img = to_device(img.unsqueeze(0), device)
    prediction =  model(img)
    _, preds = torch.max(prediction, dim = 1)
    
    
    breed = dataset.classes[preds[0].item()]
    species = ""
    if breed in dogs:
        species="dog"
    else:
        species="cat"
    return species, breed

In [ ]:
predict("<file_here>")